# "Austrian Covid data exploration"
> "Exploring the latest figures of COVID19 in Austria"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid]

# Importing the data

The data used in this post can be found at https://www.data.gv.at/covid-19/. After downloading the CSV file called `CovidFaelle_Timeline.csv`, we need to do some cleaning of the date column and split off some specific sets - the numbers for Austria, Volarlberg, Tirol and Wien. We also need to format the decimal numbers since the CSV file uses a ',' instead of a '.' as decimal.

In [1]:
#collapse
import pandas as pd

at_data = pd.read_csv("https://covid19-dashboard.ages.at/data/CovidFaelle_Timeline.csv", sep=';')
at_data['Time'] = pd.to_datetime(at_data['Time'], format='%d.%m.%Y %H:%M:%S')
at_data['SiebenTageInzidenzFaelle'] = [float('.'.join(x.split(','))) for x in at_data.SiebenTageInzidenzFaelle]
all_austria = at_data.query("BundeslandID==10").sort_values(by='Time').sort_values(by='Time')
bund_ids = [8, 7, 9, 10]
vol_tyr_wien_at = at_data.query("BundeslandID==@bund_ids")

at_data.tail()

,Time,Bundesland,BundeslandID,AnzEinwohner,AnzahlFaelle,AnzahlFaelleSum,AnzahlFaelle7Tage,SiebenTageInzidenzFaelle,AnzahlTotTaeglich,AnzahlTotSum,AnzahlGeheiltTaeglich,AnzahlGeheiltSum
3255,2021-01-16,Steiermark,6,1246395,155,46441,1587,127.3272,2,1489,232,42187
3256,2021-01-16,Tirol,7,757634,96,42770,729,96.2206,0,558,154,40873
3257,2021-01-16,Vorarlberg,8,397139,60,20722,486,122.3753,0,239,99,19732
3258,2021-01-16,Wien,9,1911191,230,77658,2199,115.0591,3,1297,5,68730
3259,2021-01-16,Österreich,10,8901064,1311,391113,11461,128.7599,15,7014,1306,358313


# Plotting the data

First we have the number of cases per 100,000 population across the whole of Austria. After a very sharp rise in cases during the end of October and the middle of November, the number of new cases have fallen due to the second lockdown. At the time of writting, the cases seem to have levelled off a little - though the Austrian government announced a third lockdown lasting a month starting from the 26th of December, in the expectation that cases will continue to rise over the festive period.

In [2]:
#collapse
import altair as alt

alt.Chart(all_austria).mark_bar(
    color='red',
    opacity=1,
    size=1
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q",
            axis=alt.Axis(title='Cases per 100k')),
).configure_axis(grid=False).configure_view(strokeWidth=0).properties(
    title='Number of cases per 100,000 in Austria', width=800
)

alt.Chart(...)

In [3]:
#collapse
bars = alt.Chart(all_austria).mark_bar(
    color='red',
    opacity=1,
    size=2
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("AnzahlFaelle:Q", axis=alt.Axis(title='New cases')),
)

line = alt.Chart(all_austria).mark_line(
    color='blue',
    opacity=0.75,
    size=1.5,
).transform_window(
    rolling_mean='mean(AnzahlFaelle)',
    frame=[0, 7]
).encode(x='yearmonthdate(Time):T',
         y='rolling_mean:Q'
         ).properties(title='New cases per day with rolling mean', width=800)

alt.layer(line, bars, data=all_austria)

alt.LayerChart(...)

Next we have the 7 day incidence rate for states of Vorarlberg, Tirol and Wien compared to all of Austria.

In [4]:
#collapse
bars = alt.Chart(vol_tyr_wien_at).mark_line().encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q", axis=alt.Axis(title='Incidence rate')),
    tooltip='Bundesland',
    color='Bundesland'
).properties(title='7 day incidence rate for Vorarlberg, Tirol, Wien and Austria', width=800).interactive()
bars
#alt.layer(bars, data=at_data).facet(alt.Row('Bundesland'), columns=1)

alt.Chart(...)

Finally we have the 

In [5]:
#collapse
states = at_data['Bundesland'].unique()
states.sort()

selection = alt.selection_single(
    name='Select',
    fields=['Bundesland'],
    init={'Bundesland': 'Vorarlberg'},
    bind={'Bundesland': alt.binding_select(options=states)}
)

# scatter plot, modify opacity based on selection
bars = alt.Chart(at_data).mark_bar().add_selection(
    selection
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q", axis=alt.Axis(title='Incidence rate')),
    tooltip='SiebenTageInzidenzFaelle:Q',
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).properties(title=f'7 day incidence rate of individual states vs rolling mean across Austria', width=800)

line = alt.Chart(all_austria).mark_line(
    color='red',
    size=2,
).transform_window(
    rolling_mean='mean(SiebenTageInzidenzFaelle)',
    frame=[0, 7]
).encode(
    x='yearmonthdate(Time):T',
    y='rolling_mean:Q'
)

alt.layer(bars, line)

alt.LayerChart(...)

Choose from the above dropdown menu to view the different states.